In [5]:
import pandas as pd
df = pd.read_csv('./data/data.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df['activated_date'] = pd.to_datetime(df['activated_date'])
df['last_payment_date'] = pd.to_datetime(df['last_payment_date'])
# I assume that days<0 means that it is a reactivated credit
df['days'] = (df['last_payment_date'] - df['activated_date']).dt.days
df = df.drop(['cust_id','activated_date','last_payment_date','oneoff_purchases','purchases_installments_frequency'], axis=1)
X = df.drop(['fraud'], axis=1)
y = df['fraud']
# X.head(5)


In [13]:
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import numpy as np

# Crear el modelo
# model = LogisticRegression(C=0.1, penalty='l1', max_iter=100, 
#                            solver='liblinear', random_state=42, class_weight='balanced')

model = LogisticRegression()
impute = SimpleImputer(strategy='median')
X_impute = impute.fit_transform(X) 
X = pd.DataFrame(X_impute, columns=X.columns, index=X.index)
# Obtener la curva de aprendizaje. train_sizes=np.linspace(0.1, 1.0, 10) asegura una distribución uniforme
# de tamaños entre el 10% y el 100% del conjunto de entrenamiento total
train_sizes, train_scores, validation_scores = learning_curve(
    model, X, y, train_sizes=np.linspace(0.1, 1.0, 10), cv=10, scoring='f1', n_jobs=-1 )

train_scores_mean = train_scores.mean(axis=1)
validation_scores_mean = validation_scores.mean(axis=1)
print(train_scores_mean)
print(validation_scores_mean)


/home/cesar/Documentos/stori_test/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/cesar/Documentos/stori_test/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

[0.72243735 0.78081689 0.7091546  0.71211757 0.71585438 0.74794495
 0.74492083 0.78020761 0.7303358  0.73854204]
[0.26628515 0.32070622 0.34219257 0.37992386 0.4705543  0.59925241
 0.62521812 0.67055833 0.68086136 0.65555833]


In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_sizes, train_scores_mean, label='f1 de Entrenamiento')
plt.plot(train_sizes, validation_scores_mean, label='f1 de Validación')
plt.ylabel('Valor del f1')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.title('Curva de Aprendizaje')
plt.legend()
plt.show()

In [19]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.metrics import mean_squared_error


base_model = Ridge()

# Bagging sobre el modelo base con subconjuntos de features
bagging_model = BaggingRegressor( # BaggingClassifier si es clasificacion
    base_estimator=base_model,
    max_features=0.8,  # Proporción de features para cada subconjunto
    max_samples=0.8,   # Proporción de datos de muestra
    random_state=42,
    n_estimators=10    # Numero de regresores ridge
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
best = bagging_model.fit(X_train, y_train)
y_pred = best.predict(X_test)
mean_squared_error(y_test, y_pred)

/home/cesar/Documentos/stori_test/venv/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.0064678172033775985

In [20]:
y_test

7669    0
6125    0
7028    0
3019    0
3479    0
       ..
1928    0
3817    0
8156    0
7691    0
5026    0
Name: fraud, Length: 1790, dtype: int64